In [1]:
# import required packages
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from datetime import datetime, timedelta

In [24]:
def extract_links_by_class(url):
    driver=webdriver.Chrome()
    driver.get(url)
    driver.maximize_window()

    driver.find_element(By.TAG_NAME,"body").send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

    view_all = driver.find_elements(By.XPATH,"//a[@class='OfferSection__ViewAllText-sc-16xojcc-1 eVcjqm']")
    ref=view_all[1].get_attribute('href')
    time.sleep(2)
    driver.get(ref)

    links = driver.find_elements(By.XPATH,"//a[@class='D113_link']")
    state_route_links = []
    for link in links:
        state = link.text
        href = link.get_attribute("href")
        if href:
            route_links = href
        state_route_links.append((state, route_links))
    driver.quit()
    return state_route_links

url = "https://www.redbus.in/"

route_links  = extract_links_by_class(url)

In [3]:
state = {'PEPSU (Punjab)':"Punjab",
         'Himachal Pradesh Tourism Development Corporation (HPTDC)':"Himachal Pradesh",
         'RSRTC':"Rajasthan",
         'PEPSU (Punjab)':"Punjab",
         'UPSRTC':"uttar pradesh",
         'Chandigarh Transport Undertaking (CTU)':"Chandigarh",
         'HRTC':"Himachal Pradesh",
         'BSRTC Operated By VIP Travels':"Kathmandu",
         'South Bengal State Transport Corporation (SBSTC)':"South Bengal",
         'NORTH BENGAL STATE TRANSPORT CORPORATION':"North BenGal",
         'Bihar State Tourism Development Corporation (BSTDC)':"Bihar",
         'WBTC (CTC)':"West Bengal",
         'West Bengal Transport Corporation':"West Bengal",
         'Bihar State Road Transport Corporation (BSRTC)':"Bihar",
         'Sikkim Nationalised Transport (SNT)':"Sikkim",
         'Assam State Transport Corporation (ASTC)':"Assam",
         'KAAC TRANSPORT':"Assam",
         'Meghalaya Transport Corporation(MTC)':"Meghalaya",
         'Kadamba Transport Corporation Limited (KTCL)':"Goa",
         'TNSTC':"Tamil Nadu",
         'APSRTC':"Andhra Pradesh",
         'Puducherry Road Transport Corporation (PRTC)':"Puducherry",
         'TSRTC':"Telangana",
         'KSRTC (Kerala)':"Kerala",
         'GSRTC':"Gujarat",
         }
state_rout_link = [] 
for i, k in route_links:
    state_rout_link.append((state[i],k))
        

In [5]:
def extract_links_by_class(state, url, class_name):
    driver = webdriver.Chrome()
    driver.get(url)
    driver.maximize_window()

    routes = []
    def route_function(): 
        route_elements = driver.find_elements(By.CLASS_NAME, class_name)
        routes_ = [] 
        for route_element in route_elements:
            route_ = route_element.text  
            route_link_ = route_element.get_attribute('href')
            routes_.append((state, route_, route_link_))
        return routes_

    try:
        body_element = driver.find_element(By.TAG_NAME, "body") 
        for _ in range(3):
            body_element.send_keys(Keys.PAGE_DOWN)
            time.sleep(1)          
        view_buses_button = driver.find_elements(By.CLASS_NAME, 'DC_117_pageTabs ')

        for i in range(len(view_buses_button)):
            if view_buses_button:
                view_buses_button[i].click()
                routes.append(route_function())
            else:
                print("No button found")

        driver.quit()
            
    except Exception as e:
        print("Error occurred:", e)
        driver.quit()
    return routes

# Example usage
routes = []
for state, link in state_rout_link:
    div_locator_1 = "route"
    routes.append(extract_links_by_class(state, link, div_locator_1))


In [7]:
dic = {}
bus_routes = []
for i in routes:
    for j in i:
        for k,l,p in j:
            if k not in dic:
                dic[k] = 1
            else:
                dic[k] += 1
            if dic[k]<= 10:
                  bus_routes.append((k,l,p))

len(bus_routes)

160

In [10]:
"""def convert_to_datetime(time_str, reference_date):
    try:
        dt = datetime.strptime(time_str, '%H:%M').replace(year=reference_date.year, month=reference_date.month, day=reference_date.day)
        return dt
    except ValueError:
        return None"""

count = 0
bus_details = []
for state, route, route_link in bus_routes:
            try:
                driver = webdriver.Chrome()
                driver.get(route_link)
                driver.maximize_window()
                time.sleep(5)
                view_buses_button = driver.find_element(By.CLASS_NAME, 'button')
                view_buses_button.click()
                time.sleep(1)
               # last_height = driver.execute_script("return document.body.scrollHeight")
               # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                body_element = driver.find_element(By.TAG_NAME, "body") 
                for _ in range(20):
                    body_element.send_keys(Keys.PAGE_DOWN)          
                time.sleep(2)  # Wait to load the page
                # new_height = driver.execute_script("return document.body.scrollHeight")
                #if new_height == last_height:
                #    break
                #last_height = new_height
            except:
                pass 
            try:
                bus_elements = driver.find_elements(By.CSS_SELECTOR, "div.bus-item")
            except:
                print("No bus elements found")
                continue
            for bus in bus_elements:
                try:
                    busname = bus.find_element(By.CSS_SELECTOR, "div.travels.lh-24.f-bold.d-color").text
                except:
                    busname = "N/A"

                try:
                    bustype = bus.find_element(By.CSS_SELECTOR, "div.bus-type.f-12.m-top-16.l-color.evBus").text
                except:
                    bustype = "N/A"

                try:
                    departing_time = bus.find_element(By.CSS_SELECTOR, "div.dp-time.f-19.d-color.f-bold").text
                    # departing_time_dt = convert_to_datetime(departing_time, datetime.now())
                except:
                    departing_time_dt = None

                try:
                    duration = bus.find_element(By.CSS_SELECTOR, "div.dur.l-color.lh-24").text
                except:
                    duration = "N/A"
                try:
                    reaching_time = bus.find_element(By.CSS_SELECTOR, "div.bp-time.f-19.d-color.disp-Inline").text
                    # reaching_time_dt = convert_to_datetime(reaching_time, datetime.now())
                    if reaching_time_dt and departing_time_dt and reaching_time_dt < departing_time_dt:
                        reaching_time_dt += timedelta(days=1)
                except:
                    reaching_time_dt = None

                try:
                    star_rating = bus.find_element(By.CSS_SELECTOR, "div.rating-sec.lh-24").text
                    star_rating = float(star_rating) if star_rating != "N/A" else 0.0
                except:
                    star_rating = 0.0

                try:
                    price = bus.find_element(By.CSS_SELECTOR, "span.f-19.f-bold").text
                    price = float(price.replace('₹', '').replace(',', '').strip()) if price != "N/A" else None
                except:
                    price = None

                try:
                    try:
                        seats_available = bus.find_element(By.CSS_SELECTOR, "div.seat-left.m-top-16").text
                    except:
                        seats_available = bus.find_element(By.CSS_SELECTOR, "div.seat-left.m-top-30").text

                    seats_available = int(seats_available.split()[0]) if seats_available != "N/A" else 0
                except:
                        seats_available = 0
                bus_details.append((route, route_link, busname, bustype, departing_time, duration, reaching_time, star_rating, price, seats_available, state))
            count+=1
            print(count)



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160


In [15]:
import pandas as pd
columns = ['route', 'route_link', 'busname', 'bustype','departing_time','duration','reaching_time','star_rating','price','seats_available', 'state']
redbus_data =  pd.DataFrame(bus_details,columns=columns)

In [16]:
From = []
To = []
location_data = redbus_data['route']
for i in location_data:
    From_1, To_1 = i.split(" to ")
    From.append(From_1)
    To.append(To_1)
redbus_data["From"] = From
redbus_data["To"] = To

In [17]:
redbus_data

,route,route_link,busname,bustype,departing_time,duration,reaching_time,star_rating,price,seats_available,state,From,To
0,Patiala to Delhi,https://www.redbus.in/bus-tickets/patiala-to-d...,PEPSU (Punjab) - 66648,Volvo AC Seater (2+2),09:50,05h 01m,14:51,4.2,692.0,32,Punjab,Patiala,Delhi
1,Patiala to Delhi,https://www.redbus.in/bus-tickets/patiala-to-d...,PEPSU (Punjab) - 66649,Volvo AC Seater (2+2),12:40,06h 09m,18:49,4.3,692.0,30,Punjab,Patiala,Delhi
2,Patiala to Delhi,https://www.redbus.in/bus-tickets/patiala-to-d...,PEPSU (Punjab) - 67101,AC Seater Hvac 2+2,15:25,05h 01m,20:26,2.6,440.0,40,Punjab,Patiala,Delhi
3,Patiala to Delhi,https://www.redbus.in/bus-tickets/patiala-to-d...,PEPSU (Punjab) - 67142,Volvo AC Seater (2+2),16:00,05h 25m,21:25,4.1,692.0,31,Punjab,Patiala,Delhi
4,Patiala to Delhi,https://www.redbus.in/bus-tickets/patiala-to-d...,PEPSU (Punjab) - 66687,Volvo AC Seater (2+2),18:10,05h 25m,23:35,3.2,692.0,38,Punjab,Patiala,Delhi
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4511,Kozhikode to Thrissur,https://www.redbus.in/bus-tickets/kozhikode-to...,Surya Connect,Volvo A/C Semi Sleeper (2+2),23:55,03h 40m,03:35,4.3,630.0,31,Kerala,Kozhikode,Thrissur
4512,Kozhikode to Thrissur,https://www.redbus.in/bus-tickets/kozhikode-to...,Cochin Express,A/C Seater Push Back (2+2),23:55,03h 40m,03:35,4.2,700.0,41,Kerala,Kozhikode,Thrissur
4513,Kozhikode to Thrissur,https://www.redbus.in/bus-tickets/kozhikode-to...,Cochin Express,Volvo Multi-Axle B9R A/c Semi Sleeper (2+2),23:59,03h 31m,03:30,4.2,700.0,35,Kerala,Kozhikode,Thrissur
4514,Kozhikode to Thrissur,https://www.redbus.in/bus-tickets/kozhikode-to...,Kalpaka Travels 1,A/C Sleeper (2+1),23:59,03h 46m,03:45,4.0,800.0,28,Kerala,Kozhikode,Thrissur


In [13]:
redbus_data.to_csv("redbus_data.csv")

In [27]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
import mysql.connector

# Your MySQL connection
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
)

# Create a SQLAlchemy engine
engine = create_engine('mysql+mysqlconnector://root:@localhost/redbus_data')

try:
    redbus_data.to_sql('route_data', con=engine, if_exists='replace', index=False)
    print("Data uploaded successfully!")

except SQLAlchemyError as e:
    print(f"An error occurred: {e}")





Data uploaded successfully!


In [28]:
redbus_data

,route,route_link,busname,bustype,departing_time,duration,reaching_time,star_rating,price,seats_available,state,From,To
0,Patiala to Delhi,https://www.redbus.in/bus-tickets/patiala-to-d...,PEPSU (Punjab) - 66648,Volvo AC Seater (2+2),09:50,05h 01m,14:51,4.2,692.0,32,Punjab,Patiala,Delhi
1,Patiala to Delhi,https://www.redbus.in/bus-tickets/patiala-to-d...,PEPSU (Punjab) - 66649,Volvo AC Seater (2+2),12:40,06h 09m,18:49,4.3,692.0,30,Punjab,Patiala,Delhi
2,Patiala to Delhi,https://www.redbus.in/bus-tickets/patiala-to-d...,PEPSU (Punjab) - 67101,AC Seater Hvac 2+2,15:25,05h 01m,20:26,2.6,440.0,40,Punjab,Patiala,Delhi
3,Patiala to Delhi,https://www.redbus.in/bus-tickets/patiala-to-d...,PEPSU (Punjab) - 67142,Volvo AC Seater (2+2),16:00,05h 25m,21:25,4.1,692.0,31,Punjab,Patiala,Delhi
4,Patiala to Delhi,https://www.redbus.in/bus-tickets/patiala-to-d...,PEPSU (Punjab) - 66687,Volvo AC Seater (2+2),18:10,05h 25m,23:35,3.2,692.0,38,Punjab,Patiala,Delhi
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4511,Kozhikode to Thrissur,https://www.redbus.in/bus-tickets/kozhikode-to...,Surya Connect,Volvo A/C Semi Sleeper (2+2),23:55,03h 40m,03:35,4.3,630.0,31,Kerala,Kozhikode,Thrissur
4512,Kozhikode to Thrissur,https://www.redbus.in/bus-tickets/kozhikode-to...,Cochin Express,A/C Seater Push Back (2+2),23:55,03h 40m,03:35,4.2,700.0,41,Kerala,Kozhikode,Thrissur
4513,Kozhikode to Thrissur,https://www.redbus.in/bus-tickets/kozhikode-to...,Cochin Express,Volvo Multi-Axle B9R A/c Semi Sleeper (2+2),23:59,03h 31m,03:30,4.2,700.0,35,Kerala,Kozhikode,Thrissur
4514,Kozhikode to Thrissur,https://www.redbus.in/bus-tickets/kozhikode-to...,Kalpaka Travels 1,A/C Sleeper (2+1),23:59,03h 46m,03:45,4.0,800.0,28,Kerala,Kozhikode,Thrissur
